In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Oct_dict_JSON.json', 'r') as json_file:
    link_day_minute_Oct_dict_JSON = json.load(json_file)

In [10]:
week_day_Oct_list = [1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 15, 16, 17, 18, 19, 22, 23, 24, 25, 26, 29, 30, 31]

In [11]:
link_day_minute_Oct_list = []
for link_idx in range(24):
    for day in week_day_Oct_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Oct_list.append(link_day_minute_Oct_dict_JSON[key] ['MD_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Oct_list)

66240

In [13]:
x = np.matrix(link_day_minute_Oct_list)
x = np.matrix.reshape(x, 24, 2760)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2760)

In [15]:
x[:,:2]

matrix([[  619406.29887901,   622635.33294238],
        [  653954.55158388,   652581.4443882 ],
        [ 1168889.43248873,  1179669.38303536],
        [ 1238849.0965825 ,  1220846.3695678 ],
        [  320553.77147025,   322388.74491494],
        [  383997.45174053,   390009.3392451 ],
        [  755348.05389466,   755761.70231304],
        [  782328.25844677,   789698.3867848 ],
        [  723664.60739773,   728581.2545577 ],
        [  739863.98839856,   753295.87091122],
        [  680020.61036128,   738856.51736715],
        [  598594.92902604,   605085.73830962],
        [  184930.20122724,   174353.96266184],
        [  188448.66627756,   185935.71953199],
        [  309372.17909778,   314595.08122636],
        [  284342.20591404,   274747.46463815],
        [  405462.60570759,   404805.82686144],
        [  404924.00302022,   409569.3941075 ],
        [  652600.91020055,   651379.20712513],
        [  593610.24675752,   589208.03652411],
        [  966093.42975912,   981562.564

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [7e-02, 1e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.12548084e+07  0.00000000e+00  5.28e+03 1.91e+03  1.00e+06     0s
   1  -7.24608609e+06 -3.86878270e+06  1.48e+03 5.37e+02  3.14e+05     0s
   2  -5.09150889e+06 -8.66026591e+06  1.09e+02 3.94e+01  9.62e+04     0s
   3  -5.73218337e+06 -6.23072402e+06  2.28e+00 8.26e-01  9.4

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Oct_weekday_MD.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[3.232708647496154e-08,
 196621.54649742073,
 151758.6320182256,
 28100.73174502886,
 75596.68148816073,
 7.651219058306012e-05,
 7.824279556614125e-05,
 197213.75537750166,
 2.0016554397828277e-09,
 121484.1626251916,
 2.001656439107071e-09,
 2.0016556536559337e-09,
 2.0016029210171914e-09,
 2.0150970167877725e-09,
 374457.84552244935,
 784.3465031757107,
 1.6893925879104643e-05,
 27290.448675230276,
 145384.62090776543,
 478449.1233472361,
 7.750451396471535e-05,
 0.0004984630941945196,
 480732.97466810787,
 3.519317584573011e-05,
 28711.263888455782,
 20090.835123493405,
 4.3315477040182246e-05,
 209895.64424994786,
 197702.17132302016,
 780.2796488063987,
 3.2751251977990365e-05,
 73910.18547214135,
 148602.90643226524,
 1.0792327472790991e-06,
 4.264663306329224e-05,
 0.00010948306384882646,
 2.144096028354925e-05,
 178455.9365772239,
 1.0659374018355723e-05,
 195154.0382589844,
 76300.13148983486,
 22507.371190780817,
 3.335213177072319e-05,
 2.2923129754682365e-05,
 400176.40570